In [1]:
import pandas as pd

from StateMachines.TradingStateMachine import TradingStateMachine
from StrategyBuilders import getStrategyBuilder
from Strategies.AbstractTradingStrategy import AbstractTradingStrategy

In [2]:
folder = "./Data/bybit"
folder_lab1 = "./Data/lab1"
file = "ETH-USDT:USDT_1m.csv"
periods = [5, 10, 15, 20, 30, 60, 120]

In [3]:
folder = "./Data/gemini"
folder_lab1 = "./Data/lab1"
file = "gemini_1m_features.csv"


In [4]:

strategy_builder = getStrategyBuilder('MODEL')
trader: TradingStateMachine = TradingStateMachine(strategy_builder("gemini_btcusd_1m"))

df = pd.read_csv(f"{folder}/{file}")
len(df)
print(list(df.columns))
# ['Unnamed: 0', 'timestamp', 'symbol', 'open', 'high', 'low', 'close', 'volume', 'stoch_rsi', 'stoch_rsi_k', 'stoch_rsi_d', 'stoch_rsi_overbought_', 'stoch_rsi_oversold_', 'stoch_rsi_bullish_crossover_', 'stoch_rsi_bearish_crossover_', 'stoch_rsi_slope', 'stoch_rsi_k_slope', 'stoch_rsi_d_slope', 'stoch_rsi_trend_', 'stoch_rsi_change_of_trend_', 'hma200', 'hma200_slope', 'hma200_trend_', 'hma200_change_of_trend_', 'hma200_above_price_', 'macd', 'macd_signal', 'macd_diff', 'prev_macd_diff', 'macd_bullish_crossover_', 'macd_bearish_crossover_', 'macd_positive_', 'macd_slope', 'macd_signal_slope', 'macd_trend_', 'macd_change_of_trend_']

['Unnamed: 0', 'timestamp', 'symbol', 'open', 'high', 'low', 'close', 'volume', 'stoch_rsi', 'stoch_rsi_k', 'stoch_rsi_d', 'stoch_rsi_overbought_', 'stoch_rsi_oversold_', 'stoch_rsi_bullish_crossover_', 'stoch_rsi_bearish_crossover_', 'stoch_rsi_slope', 'stoch_rsi_k_slope', 'stoch_rsi_d_slope', 'stoch_rsi_trend_', 'stoch_rsi_change_of_trend_', 'hma200', 'hma200_slope', 'hma200_trend_', 'hma200_change_of_trend_', 'hma200_above_price_', 'macd', 'macd_signal', 'macd_diff', 'prev_macd_diff', 'macd_bullish_crossover_', 'macd_bearish_crossover_', 'macd_positive_', 'macd_slope', 'macd_signal_slope', 'macd_trend_', 'macd_change_of_trend_']


In [5]:
period = 30
periods = [period]
first_year = 2017

In [6]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year_'] = df['timestamp'].dt.year

features_base = ['timestamp', 'close', 'year_']
columns = features_base + AbstractTradingStrategy.get_features_list(df)
features_stochrsi = [col for col in columns if 'stoch_rsi' in col and 'change_of_trend' not in col and col[-1] == '_']
features_hma = [col for col in columns if 'hma' in col and 'change_of_trend' not in col and col[-1] == '_']
features_macd = [col for col in columns if 'macd' in col and 'change_of_trend' not in col and col[-1] == '_']


In [7]:
def get_best_group(df, features, period, sort='mean', max=5, ascending=False, head=True):
    
    column = f'close_p_in_{period}'
    grouped_df = df.groupby(features)[column].agg(['count', 'mean', 'min', 'max'])
    grouped_df = grouped_df.sort_values(by=sort, ascending=ascending)
    if head:
        return grouped_df.head(max)
    else:
        return grouped_df.tail(max)

def convert_grouped_to_dataframe(grouped_df):
    df = grouped_df.reset_index()
    return df

# import pandas as pd

def append_dataframes(df1, df2):
    # Concatenate the dataframes
    df = pd.concat([df1, df2], ignore_index=True)
    return df

we want to work on the history 1m from bitcoin (gemini)

Dataframe must be indexed by timestamp

In [8]:
df = df[columns]
df = df.set_index('timestamp')

df.tail(5)

,close,year_,stoch_rsi_overbought_,stoch_rsi_oversold_,stoch_rsi_bullish_crossover_,stoch_rsi_bearish_crossover_,stoch_rsi_trend_,stoch_rsi_change_of_trend_,hma200_trend_,hma200_change_of_trend_,hma200_above_price_,macd_bullish_crossover_,macd_bearish_crossover_,macd_positive_,macd_trend_,macd_change_of_trend_,year_
timestamp,,,,,,,,,,,,,,,,,
2024-04-07 00:24:00,69046.77,2024,False,False,False,False,Downtrend,NaN,Downtrend,NaN,True,False,False,True,Uptrend,NaN,2024
2024-04-07 00:25:00,68944.63,2024,False,True,False,False,Downtrend,NaN,Downtrend,NaN,True,False,True,False,Uptrend,NaN,2024
2024-04-07 00:26:00,68942.84,2024,False,True,False,False,Downtrend,NaN,Downtrend,NaN,True,False,False,False,Downtrend,Uptrend->Downtrend,2024
2024-04-07 00:27:00,68947.39,2024,False,True,False,False,Downtrend,NaN,Downtrend,NaN,True,False,False,False,Downtrend,NaN,2024
2024-04-07 00:28:00,68890.02,2024,False,True,True,False,Downtrend,NaN,Downtrend,NaN,True,False,False,False,Downtrend,NaN,2024


Now we calculate the result for different number of intervals

In [9]:
# adding shifted periods results
features_close = [f'close_in_{period}' for period in periods]
features_close_perc = [f'close_p_in_{period}' for period in periods]
for period in periods:
    column_name = f'close_in_{period}'
    df[column_name] = df['close'].shift(-period)
    column_perc = f'close_p_in_{period}'
    df[column_perc] = ((df['close'].shift(-period) - df['close']) / df['close']) * 100
    
df.to_csv(f"{folder_lab1}/{file}_periods.csv")
df.columns

Index(['close', 'year_', 'stoch_rsi_overbought_', 'stoch_rsi_oversold_',
       'stoch_rsi_bullish_crossover_', 'stoch_rsi_bearish_crossover_',
       'stoch_rsi_trend_', 'stoch_rsi_change_of_trend_', 'hma200_trend_',
       'hma200_change_of_trend_', 'hma200_above_price_',
       'macd_bullish_crossover_', 'macd_bearish_crossover_', 'macd_positive_',
       'macd_trend_', 'macd_change_of_trend_', 'year_', 'close_in_30',
       'close_p_in_30'],
      dtype='object')

we classify the features based on the result for the given number of intervals

In [10]:
# period = 30
x = df[df[f'close_p_in_{period}'] > 1]
x[['year_', 'close', f'close_in_{period}', f'close_p_in_{period}']].head(10)

,year_,year_,close,close_in_30,close_p_in_30
timestamp,,,,,
2017-01-01 16:11:00,2017,2017,982.99,994.95,1.216696
2017-01-01 16:12:00,2017,2017,982.99,996.32,1.356067
2017-01-01 16:13:00,2017,2017,982.99,996.32,1.356067
2017-01-01 16:14:00,2017,2017,982.99,1000.01,1.731452
2017-01-01 16:15:00,2017,2017,982.99,1000.00,1.730435
2017-01-01 16:16:00,2017,2017,982.97,999.99,1.731487
2017-01-01 16:17:00,2017,2017,982.51,996.71,1.445278
2017-01-01 16:18:00,2017,2017,979.34,998.70,1.976842
2017-01-01 16:19:00,2017,2017,979.34,998.70,1.976842


In [11]:
# Create a column variable that is the combination of all the features_ lists
all_features = features_stochrsi + features_hma + features_macd

# removing likely not valid rows
features = all_features + [f'close_p_in_{period}']
new_df = df[features].iloc[200:-period]

len(new_df)


3680356

We reduce the dataframe to the last 4 years

In [12]:

new_df = new_df[new_df.index.year >= first_year]
len(new_df)

3680356

1. Chi-Square Test for Independence
The Chi-Square test is useful for testing the relationship between two categorical variables. It can tell you whether the distribution of sample categorical data matches an expected distribution.

Use Case: To determine if there is a significant association between two categorical features.

In [13]:
import numpy as np
from scipy.stats import chi2_contingency

# Assuming df is your DataFrame and 'feature' and 'target' are your categorical columns
def chi2_test(df, feature, target):
    contingency_table = pd.crosstab(df[feature], df[target])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    return chi2, p

def chi2_feature_selection(df, features, target):
    results = []
    for feature in features:
        chi2, p = chi2_test(df, feature, target)
        results.append({'feature': feature, 'chi2': chi2, 'p-value': p})
    return results

print(new_df.columns)





Index(['stoch_rsi_overbought_', 'stoch_rsi_oversold_',
       'stoch_rsi_bullish_crossover_', 'stoch_rsi_bearish_crossover_',
       'stoch_rsi_trend_', 'hma200_trend_', 'hma200_above_price_',
       'macd_bullish_crossover_', 'macd_bearish_crossover_', 'macd_positive_',
       'macd_trend_', 'close_p_in_30'],
      dtype='object')


Selecting the best features for BUY signal

In [14]:
print(new_df.columns)
results = []
new_df['target'] = np.where(new_df[f'close_p_in_{period}'] > 0, 'Increase', 'Decrease')
for feature in all_features:
    contingency_table = pd.crosstab(new_df[feature], new_df['target'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    results.append({'feature': feature, 'chi2': chi2, 'p-value': p})

# Sort the results by chi2 in descending order
results = sorted(results, key=lambda x: x['chi2'], reverse=True)

# Display the sorted results
for result in results:
    print(f"Feature: {result['feature']}, Chi-Square Value: {result['chi2']}, P-Value: {result['p-value']}")


Index(['stoch_rsi_overbought_', 'stoch_rsi_oversold_',
       'stoch_rsi_bullish_crossover_', 'stoch_rsi_bearish_crossover_',
       'stoch_rsi_trend_', 'hma200_trend_', 'hma200_above_price_',
       'macd_bullish_crossover_', 'macd_bearish_crossover_', 'macd_positive_',
       'macd_trend_', 'close_p_in_30'],
      dtype='object')
Feature: hma200_trend_, Chi-Square Value: 12353.395693580394, P-Value: 0.0
Feature: macd_trend_, Chi-Square Value: 6316.399035133662, P-Value: 0.0
Feature: hma200_above_price_, Chi-Square Value: 4396.84908636442, P-Value: 0.0
Feature: macd_positive_, Chi-Square Value: 934.1688806297338, P-Value: 3.6649553689830154e-205
Feature: stoch_rsi_oversold_, Chi-Square Value: 592.7144643286736, P-Value: 6.433940516481268e-131
Feature: stoch_rsi_overbought_, Chi-Square Value: 309.18592665406055, P-Value: 3.2852303811670306e-69
Feature: stoch_rsi_trend_, Chi-Square Value: 18.80046264476158, P-Value: 8.270493184195254e-05
Feature: macd_bearish_crossover_, Chi-Square Valu

In [18]:
cluster1 = ['hma200_trend_','hma200_above_price_','macd_trend_']
best_df = get_best_group(new_df, cluster1, period)

save_model = True
if save_model:
    print("saving model")
    x = convert_grouped_to_dataframe(best_df)
    x = x[cluster1 ]
    x['signal'] = 'Buy'
    x.to_csv(f"./simple_model_buy.csv")

best_df

saving model


count      mean        min  \
hma200_trend_ hma200_above_price_ macd_trend_                                 
Not trending  False               Uptrend          255  0.027985  -1.556668   
              True                Not trending     961  0.018049  -3.434768   
Downtrend     True                Downtrend     633405  0.014052 -38.867640   
Not trending  False               Downtrend        131  0.013730  -2.950425   
Uptrend       False               Downtrend     631711  0.011664 -17.231202   

                                                      max  
hma200_trend_ hma200_above_price_ macd_trend_              
Not trending  False               Uptrend        1.900347  
              True                Not trending   3.211990  
Downtrend     True                Downtrend     28.299313  
Not trending  False               Downtrend      1.583724  
Uptrend       False               Downtrend     16.114799

In [16]:
print(new_df.columns)
results = []
new_df['target'] = np.where(new_df[f'close_p_in_{period}'] < 0, 'Decrease', 'Increase')
for feature in all_features:
    contingency_table = pd.crosstab(new_df[feature], new_df['target'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    results.append({'feature': feature, 'chi2': chi2, 'p-value': p})

# Sort the results by chi2 in descending order
results = sorted(results, key=lambda x: x['chi2'], reverse=True)

# Display the sorted results
for result in results:
    print(f"Feature: {result['feature']}, Chi-Square Value: {result['chi2']}, P-Value: {result['p-value']}")

Index(['stoch_rsi_overbought_', 'stoch_rsi_oversold_',
       'stoch_rsi_bullish_crossover_', 'stoch_rsi_bearish_crossover_',
       'stoch_rsi_trend_', 'hma200_trend_', 'hma200_above_price_',
       'macd_bullish_crossover_', 'macd_bearish_crossover_', 'macd_positive_',
       'macd_trend_', 'close_p_in_30', 'target'],
      dtype='object')
Feature: hma200_trend_, Chi-Square Value: 11721.199560272733, P-Value: 0.0
Feature: macd_trend_, Chi-Square Value: 5589.0769460894735, P-Value: 0.0
Feature: hma200_above_price_, Chi-Square Value: 4052.3575823272135, P-Value: 0.0
Feature: stoch_rsi_trend_, Chi-Square Value: 1256.4634481107378, P-Value: 1.4535463542913182e-273
Feature: macd_positive_, Chi-Square Value: 994.8406698458996, P-Value: 2.375314922912583e-218
Feature: stoch_rsi_overbought_, Chi-Square Value: 439.3227440277411, P-Value: 1.519999599636694e-97
Feature: stoch_rsi_oversold_, Chi-Square Value: 384.2748931820654, P-Value: 1.4596990605353076e-85
Feature: stoch_rsi_bearish_crossover

In [19]:
cluster1 = ['hma200_trend_','hma200_above_price_','macd_trend_']
best_df = get_best_group(new_df, cluster1, period, ascending=False, head=False)

save_model = True
if save_model:
    print("saving model")
    x = convert_grouped_to_dataframe(best_df)
    x = x[cluster1 ]
    x['signal'] = 'Sell'
    x.to_csv(f"./simple_model_sell.csv")

best_df

saving model


count      mean       min  \
hma200_trend_ hma200_above_price_ macd_trend_                               
Downtrend     True                Not trending   3907 -0.013276 -4.064648   
Uptrend       False               Not trending   3051 -0.014244 -2.578498   
Not trending  True                Downtrend       267 -0.014494 -1.796370   
              False               Not trending   3652 -0.058708 -6.602463   
Uptrend       True                Not trending    760 -0.115130 -2.646651   

                                                     max  
hma200_trend_ hma200_above_price_ macd_trend_             
Downtrend     True                Not trending  1.501541  
Uptrend       False               Not trending  2.672373  
Not trending  True                Downtrend     1.230483  
              False               Not trending  3.898413  
Uptrend       True                Not trending  1.088616